In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!apt-get update && apt-get install -y libzbar0

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:4 https://cli.github.com/packages stable InRelease [3,917 B]               
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]      
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,637 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,289 kB]
Get:13 https:

In [3]:
!pip install --no-cache-dir \
  colpali-engine \
  chromadb \
  rank_bm25 \
  langchain \
  langchain-community \
  langchain-huggingface \
  langchain-groq \
  langchain-experimental \
  langgraph \
  pymupdf \
  pyzbar \
  accelerate \
  bitsandbytes \
  peft \
  pdf2image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 258.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-huggingface to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-groq to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-experimental to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langgraph-prebuilt to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langgraph-prebuilt to determine which version is compatible with other requirements. This

In [9]:
import os
import torch
import fitz  # PyMuPDF
import numpy as np
import warnings
import chromadb
from typing import List, TypedDict, Any, Dict
from PIL import Image as PILImage
from pyzbar.pyzbar import decode
from tqdm.notebook import tqdm

# LangChain / LangGraph
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_huggingface import HuggingFaceEmbeddings 
from langgraph.graph import StateGraph, END
from langchain_experimental.text_splitter import SemanticChunker

# Vision Models
from colpali_engine.models import ColPali, ColPaliProcessor
from transformers import BitsAndBytesConfig

warnings.filterwarnings("ignore")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Hardware Accelerator: {DEVICE}")

# --- API KEY SETUP ---
# Ensure your Groq API Key is set here or in Kaggle Secrets
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    os.environ["GROQ_API_KEY"] = user_secrets.get_secret("GROQ_API_KEY")
    print("✅ GROQ_API_KEY loaded from Secrets.")
except:
    # Manual Fallback if not using Kaggle Secrets
    if "GROQ_API_KEY" not in os.environ:
        os.environ["GROQ_API_KEY"] = "YOUR_GROQ_API_KEY_HERE"
        print("⚠️ Used manual API key placeholder. Replace if needed.")

✅ Hardware Accelerator: cuda
✅ GROQ_API_KEY loaded from Secrets.


In [4]:
import os
import torch
import fitz  # PyMuPDF
import numpy as np
import warnings
import chromadb
from typing import List, TypedDict, Any, Dict
from PIL import Image as PILImage
from pyzbar.pyzbar import decode
from tqdm.notebook import tqdm

# LangChain / LangGraph
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_community.retrievers import BM25Retriever
from langchain_huggingface import HuggingFaceEmbeddings 
from langgraph.graph import StateGraph, END
from langchain_experimental.text_splitter import SemanticChunker

# Vision Models
from colpali_engine.models import ColPali, ColPaliProcessor
from transformers import BitsAndBytesConfig

warnings.filterwarnings("ignore")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- 1. ENGINE CLASS (Fixes InvalidArgumentError) ---
class ByteMeEngine:
    def __init__(self):
        # A. Vision Model (ColPali)
        print(">> Loading ColPali (Vision Model)...")
        bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16)
        self.colpali_model = ColPali.from_pretrained("vidore/colpali-v1.2", quantization_config=bnb_config, torch_dtype=torch.float16, device_map="auto")
        self.colpali_processor = ColPaliProcessor.from_pretrained("vidore/colpali-v1.2")
        
        # B. Text Embedder (MiniLM - 384 dim)
        self.dense_embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={'device': DEVICE})
        self.text_splitter = SemanticChunker(self.dense_embedder)

        # C. ChromaDB: SEPARATE COLLECTIONS
        self.db_path = "/kaggle/working/chroma_db"
        self.client = chromadb.PersistentClient(path=self.db_path)
        
        # Collection 1: Text (384 dimensions)
        self.text_collection = self.client.get_or_create_collection(
            name="text_store", 
            metadata={"hnsw:space": "cosine"}
        )
        
        # Collection 2: Vision (ColPali dimensions)
        self.vision_collection = self.client.get_or_create_collection(
            name="vision_store", 
            metadata={"hnsw:space": "cosine"}
        )
        
        # D. Sparse Retriever
        self.bm25_retriever = None

    def process_pdf(self, pdf_path: str):
        print(f"🚀 Ingesting: {pdf_path}")
        doc = fitz.open(pdf_path)
        
        # Lists for Batch Processing
        vis_ids, vis_vecs, vis_metas, vis_docs = [], [], [], []
        txt_ids, txt_vecs, txt_metas, txt_docs = [], [], [], []
        full_text_list = []

        # 1. Vision Processing
        for page_num, page in enumerate(tqdm(doc, desc="Vision Encoding")):
            # Image & QR
            pix = page.get_pixmap(dpi=150)
            img_np = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
            pil_img = PILImage.fromarray(img_np)
            qr_data = [obj.data.decode('utf-8') for obj in decode(img_np)]
            qr_text = f" [QR: {qr_data}]" if qr_data else ""
            
            # Embed Image (ColPali)
            with torch.no_grad():
                batch = self.colpali_processor.process_images([pil_img]).to(DEVICE)
                emb = self.colpali_model(**batch)
                # POOLING: Average patches to get single vector
                pooled_emb = torch.mean(emb, dim=1).float().cpu().numpy()[0].tolist()

            vis_ids.append(f"vis_{page_num}")
            vis_vecs.append(pooled_emb)
            # Store minimal text in vision doc to allow retrieval verification
            content_preview = (page.get_text()[:200] + qr_text).replace("\n", " ")
            vis_metas.append({"source": pdf_path, "page": page_num+1, "type": "vision"})
            vis_docs.append(content_preview)
            full_text_list.append(page.get_text())

        # 2. Text Processing (Semantic Chunking)
        print(">> Semantic Chunking & Text Embedding...")
        text_chunks = self.text_splitter.create_documents(full_text_list)
        text_embeddings = self.dense_embedder.embed_documents([t.page_content for t in text_chunks])
        
        for i, chunk in enumerate(text_chunks):
            txt_ids.append(f"txt_{i}")
            txt_vecs.append(text_embeddings[i])
            txt_metas.append({"source": pdf_path, "type": "text"})
            txt_docs.append(chunk.page_content)

        # 3. Upsert to Respective Collections
        if vis_ids:
            print(f">> Indexing {len(vis_ids)} Vision Pages...")
            self.vision_collection.upsert(ids=vis_ids, embeddings=vis_vecs, metadatas=vis_metas, documents=vis_docs)
            
        if txt_ids:
            print(f">> Indexing {len(txt_ids)} Text Chunks...")
            self.text_collection.upsert(ids=txt_ids, embeddings=txt_vecs, metadatas=txt_metas, documents=txt_docs)

        # 4. Build BM25
        print(">> Building BM25 Index...")
        self.bm25_retriever = BM25Retriever.from_texts(full_text_list)
        self.bm25_retriever.k = 3
        print("✅ Ingestion Complete.")

    def hybrid_search(self, query: str, k=5) -> List[str]:
        """Queries Text (Dense + Sparse) and Vision collections and merges results."""
        results = []
        
        # A. Text Dense Search
        q_dense = self.dense_embedder.embed_query(query)
        txt_res = self.text_collection.query(query_embeddings=[q_dense], n_results=k)
        if txt_res['documents']: results.extend(txt_res['documents'][0])
        
        # B. Vision Dense Search (Using ColPali on Query)
        with torch.no_grad():
            # ColPali expects query as a list of strings
            batch = self.colpali_processor.process_queries([query]).to(DEVICE)
            emb = self.colpali_model(**batch)
            q_vis = torch.mean(emb, dim=1).float().cpu().numpy()[0].tolist()
            
        vis_res = self.vision_collection.query(query_embeddings=[q_vis], n_results=k)
        if vis_res['documents']: results.extend(vis_res['documents'][0])
        
        # C. BM25 Sparse Search
        if self.bm25_retriever:
            sparse_res = self.bm25_retriever.invoke(query)
            results.extend([d.page_content for d in sparse_res])
            
        # Deduplicate
        return list(set(results))

# Initialize Engine
engine = ByteMeEngine()

2026-01-15 17:21:28.611914: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768497688.942839      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768497689.037332      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768497689.845998      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768497689.846036      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768497689.846039      55 computation_placer.cc:177] computation placer alr

>> Loading ColPali (Vision Model)...


adapter_config.json:   0%|          | 0.00/750 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# 1. Auto-Find PDF
pdf_path = None
for root, _, files in os.walk("/kaggle/input"):
    for f in files:
        if f.endswith(".pdf"):
            pdf_path = os.path.join(root, f)
            break

if pdf_path:
    # 2. Ingest
    engine.process_pdf(pdf_path)
else:
    print("❌ No PDF found! Please upload a PDF to /kaggle/input.")

🚀 Ingesting: /kaggle/input/hcl-data/Annual-Report-2024-25.pdf


Vision Encoding:   0%|          | 0/423 [00:00<?, ?it/s]

	i=21 f=-1(000) part=1
	i=31 f=-1(000) part=1
	i=4 f=-1(000) part=1
	i=4 f=-1(000) part=1
	i=4 f=-1(000) part=0
	i=8 f=-1(000) part=0
	i=4 f=-1(000) part=0
	i=8 f=-1(000) part=0
	i=8 f=-1(000) part=1
	i=17 f=-1(000) part=0
	i=4 f=-1(000) part=0
	i=5 f=-1(000) part=1
	i=9 f=-1(000) part=1
	i=5 f=-1(000) part=0
	i=7 f=-1(000) part=0
	i=5 f=-1(000) part=1
	i=7 f=-1(000) part=1
	i=7 f=-1(000) part=1
	i=5 f=-1(000) part=1
	i=19 f=-1(000) part=0
	i=5 f=-1(000) part=0
	i=14 f=-1(000) part=0
	i=5 f=-1(000) part=0
	i=19 f=-1(000) part=0
	i=19 f=-1(000) part=1
	i=7 f=-1(000) part=0
	i=19 f=-1(000) part=0
	i=18 f=-1(000) part=1
	i=19 f=-1(000) part=0
	i=10 f=-1(000) part=0
	i=18 f=-1(000) part=0
	i=10 f=-1(000) part=0
	i=10 f=-1(000) part=1
	i=19 f=-1(000) part=0
	i=11 f=-1(000) part=0
	i=10 f=-1(000) part=0
	i=16 f=-1(000) part=0
	i=5 f=-1(000) part=0
	i=6 f=-1(000) part=0
	i=5 f=-1(000) part=0
	i=10 f=-1(000) part=1
	i=13 f=-1(000) part=0
	i=13 f=-1(000) part=1
	i=14 f=-1(000) part=0
	i=0 f=-1(

>> Semantic Chunking & Text Embedding...
>> Indexing 423 Vision Pages...
>> Indexing 910 Text Chunks...
>> Building BM25 Index...
✅ Ingestion Complete.

❓ User: What is the consolidated revenue growth?


NameError: name 'app' is not defined

In [19]:
# Memory-augmented LLM instances
llm_router = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
llm_gen = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.3)  # Slightly higher temp for creativity

In [13]:
import json
import ast
from langgraph.graph import StateGraph, END
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# --- 1. ROBUST PARSING HELPER ---
def parse_json_safe(text_output):
    """
    Safely parses LLM output that might use single quotes or markdown blocks.
    """
    # Clean markdown
    text = text_output.strip().replace("```json", "").replace("```", "")
    
    try:
        # Try standard JSON
        return json.loads(text)
    except json.JSONDecodeError:
        try:
            # Try Python literal eval (handles single quotes)
            return ast.literal_eval(text)
        except:
            # Fallback: simple string search
            if "yes" in text.lower(): return {"score": "yes"}
            return {"score": "no"}

# --- 2. IMPROVED PROMPTS & CHAINS ---
rag_chain = (
    PromptTemplate(template="Context: {context} \n Q: {question} \n Answer:", input_variables=["context", "question"]) 
    | llm_gen 
    | StrOutputParser()
)
# A. Rewriter
rewrite_prompt = PromptTemplate(
    template="""You are a search query optimizer. 
    Rewrite the user's question to be short, specific, and keyword-rich for a vector database.
    Do NOT output a list. Output ONLY the rewritten query string.
    
    Original: {question}
    New Query:""",
    input_variables=["question"]
)
# Use StrOutputParser (Text only)
rewriter_chain = rewrite_prompt | llm_router | StrOutputParser()

# B. Grader
grade_prompt = PromptTemplate(
    template="""You are a grader assessing relevance. 
    Does the document contain ANY information related to the user's question?
    
    Document: {document}
    Question: {question}
    
    Return strictly valid JSON: {{"score": "yes"}} or {{"score": "no"}}.""",
    input_variables=["question", "document"]
)
# Use StrOutputParser + Manual Safe Parse
grader_chain = grade_prompt | llm_router | StrOutputParser()

# C. Grounding
grounding_prompt = PromptTemplate(
    template="""Context: {context} 
    Answer: {generation} 
    
    Is the answer fully supported by the context? 
    Return strictly valid JSON: {{"score": "yes"}} or {{"score": "no"}}.""",
    input_variables=["context", "generation"]
)
# Use StrOutputParser + Manual Safe Parse
grounding_chain = grounding_prompt | llm_router | StrOutputParser()

# --- 3. GRAPH DEFINITION ---

class AgentState(TypedDict):
    question: str
    documents: List[str]
    generation: str
    is_grounded: bool
    retries: int 

def retrieve_node(state):
    print(f"🔍 Hybrid Retrieval for: '{state['question']}'")
    docs = engine.hybrid_search(state["question"])
    return {"documents": docs, "retries": state.get("retries", 0)}

def grade_node(state):
    print("📝 Grading Retrieval Quality...")
    relevant = []
    
    if not state["documents"]: 
        return {"documents": [], "retries": state["retries"] + 1}
        
    for doc in state["documents"]:
        try:
            # RAW TEXT output from LLM
            raw_res = grader_chain.invoke({"question": state["question"], "document": doc})
            # SAFE PARSE
            parsed_res = parse_json_safe(raw_res)
            
            if parsed_res.get("score") == "yes":
                relevant.append(doc)
        except: continue
    
    # Increment retries if grading failed
    new_retries = state["retries"] + 1 if not relevant else state["retries"]
    return {"documents": relevant, "retries": new_retries}

def rewrite_node(state):
    print("🔄 Retrieval Poor. Rewriting Query...")
    new_q = rewriter_chain.invoke({"question": state["question"]})
    new_q = new_q.strip().replace('"', '')
    print(f"   -> New Query: '{new_q}'")
    return {"question": new_q}

def generate_node(state):
    print("💡 Generating Answer...")
    gen = rag_chain.invoke({"context": state["documents"], "question": state["question"]})
    return {"generation": gen}

def graceful_fail_node(state):
    print("❌ Max Retries Reached. Returning fallback.")
    return {"generation": "I'm sorry, I tried several times but couldn't find relevant information in the provided document to answer your question. Please try rephrasing."}

def reflection_node(state):
    print("🛡️ Verifying Grounding...")
    if "I'm sorry" in state["generation"]:
        return {"is_grounded": False}
        
    raw_res = grounding_chain.invoke({"context": state["documents"], "generation": state["generation"]})
    parsed_res = parse_json_safe(raw_res)
    
    if parsed_res.get("score") == "no":
        return {"is_grounded": False, "generation": "I found some documents, but the answer I generated couldn't be fully verified against them."}
    return {"is_grounded": True}

# Build Graph
workflow = StateGraph(AgentState)
workflow.add_node("retrieve", retrieve_node)
workflow.add_node("grade", grade_node)
workflow.add_node("rewrite", rewrite_node)
workflow.add_node("generate", generate_node)
workflow.add_node("reflect", reflection_node)
workflow.add_node("fail", graceful_fail_node)

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "grade")

def check_relevance(state):
    if state["documents"]: return "generate"
    if state["retries"] > 2: return "fail"
    return "rewrite"

workflow.add_conditional_edges("grade", check_relevance)
workflow.add_edge("rewrite", "retrieve")
workflow.add_edge("generate", "reflect")
workflow.add_edge("reflect", END)
workflow.add_edge("fail", END)

app = workflow.compile()
print("✅ Robust Agent Graph Compiled.")

✅ Robust Agent Graph Compiled.


In [14]:
if 'engine' in globals():
    q = "What is the consolidated revenue growth?"
    print(f"\n❓ User: {q}")
    
    # Invoke the app
    res = app.invoke({"question": q, "retries": 0})
    
    print("\n" + "="*40)
    print(f"✅ Final Answer:\n{res.get('generation', 'No answer generated.')}")
    print("="*40)
else:
    print("❌ Engine not found. Please run the Ingestion cell first.")


❓ User: What is the consolidated revenue growth?
🔍 Hybrid Retrieval for: 'What is the consolidated revenue growth?'
📝 Grading Retrieval Quality...
💡 Generating Answer...
🛡️ Verifying Grounding...

✅ Final Answer:
The consolidated revenue growth is 6.5%. This is based on the increase in revenue from operations from ₹109,913 crores in the year ended 31 March 2024 to ₹117,055 crores in the year ended 31 March 2025.


In [15]:
# List of test questions targeting different capabilities (Vision, Text, Reasoning, Negative)
test_questions = [
    "What is the trend of attrition rate over the last 3 years?",
    "How is revenue distributed across different geographies?",
    "What is the recommended final dividend per share for FY25?",
    "Summarize the company's progress on its sustainability or ESG goals.",
    "What is the projected revenue for FY 2030?"  # Negative test
]

print("🚀 Starting Evaluation Run...\n")

if 'app' in globals() and 'engine' in globals():
    for i, q in enumerate(test_questions, 1):
        print(f"\n❓ Question {i}: {q}")
        
        # Invoke the Agent
        try:
            res = app.invoke({"question": q, "retries": 0})
            answer = res.get("generation", "No answer generated.")
            
            print("-" * 20)
            print(f"✅ Agent Answer:\n{answer}")
            print("-" * 40)
            
        except Exception as e:
            print(f"❌ Error: {e}")
else:
    print("❌ Agent app or Engine not found. Please ensure previous cells were run successfully.")

🚀 Starting Evaluation Run...


❓ Question 1: What is the trend of attrition rate over the last 3 years?
🔍 Hybrid Retrieval for: 'What is the trend of attrition rate over the last 3 years?'
📝 Grading Retrieval Quality...
💡 Generating Answer...
🛡️ Verifying Grounding...
--------------------
✅ Agent Answer:
The provided context is insufficient to determine the trend of the attrition rate over the last 3 years. The given information only includes a date (31 March 2024) and a percentage (7.50%), but it does not provide any data on attrition rates for the previous years. To analyze the trend, we would need more information, such as the attrition rates for 2022 and 2023.
----------------------------------------

❓ Question 2: How is revenue distributed across different geographies?
🔍 Hybrid Retrieval for: 'How is revenue distributed across different geographies?'
📝 Grading Retrieval Quality...
💡 Generating Answer...
🛡️ Verifying Grounding...
--------------------
✅ Agent Answer:
The revenue di

In [16]:
import shutil

folder_path = "/kaggle/working/chroma_db"   # or any folder
zip_path = "/kaggle/working/chroma_db.zip"

shutil.make_archive(zip_path.replace(".zip", ""), 'zip', folder_path)


'/kaggle/working/chroma_db.zip'

In [17]:
from datetime import datetime
from collections import deque
import hashlib

class MemoryManager:
    """
    Professional Memory Management System for Agentic RAG
    Implements both short-term (working memory) and long-term (persistent) memory
    """
    
    def __init__(self, chromadb_client, embedder, max_short_term=10):
        """
        Args:
            chromadb_client: ChromaDB client instance
            embedder: HuggingFace embeddings model
            max_short_term: Maximum number of recent exchanges to keep in short-term memory
        """
        self.embedder = embedder
        self.max_short_term = max_short_term
        
        # SHORT-TERM MEMORY: In-memory conversation buffer (FIFO queue)
        self.conversation_buffer = deque(maxlen=max_short_term)
        
        # LONG-TERM MEMORY: Persistent ChromaDB collections
        self.memory_collection = chromadb_client.get_or_create_collection(
            name="long_term_memory",
            metadata={"hnsw:space": "cosine"}
        )
        
        self.facts_collection = chromadb_client.get_or_create_collection(
            name="extracted_facts",
            metadata={"hnsw:space": "cosine"}
        )
        
        print("✅ Memory Manager Initialized")
        print(f"   - Short-term capacity: {max_short_term} exchanges")
        print(f"   - Long-term storage: ChromaDB (persistent)")
    
    def add_exchange(self, question: str, answer: str, metadata: dict = None):
        """
        Add a Q&A exchange to SHORT-TERM memory
        """
        exchange = {
            "question": question,
            "answer": answer,
            "timestamp": datetime.now().isoformat(),
            "metadata": metadata or {}
        }
        self.conversation_buffer.append(exchange)
        return exchange
    
    def get_recent_history(self, n: int = None) -> List[Dict]:
        """
        Retrieve recent conversation history from SHORT-TERM memory
        """
        n = n or self.max_short_term
        return list(self.conversation_buffer)[-n:]
    
    def format_history_for_prompt(self, n: int = 3) -> str:
        """
        Format recent history as a string for LLM context
        """
        history = self.get_recent_history(n)
        if not history:
            return "No previous conversation."
        
        formatted = []
        for i, exch in enumerate(history, 1):
            formatted.append(f"[{i}] Q: {exch['question']}\n    A: {exch['answer'][:150]}...")
        
        return "\n".join(formatted)
    
    def store_to_long_term(self, question: str, answer: str, importance_score: float = 0.5):
        """
        Persist important exchanges to LONG-TERM memory (ChromaDB)
        Args:
            importance_score: 0-1, higher = more important (for future filtering)
        """
        # Create unique ID based on content hash
        content = f"{question}|{answer}"
        mem_id = hashlib.md5(content.encode()).hexdigest()
        
        # Embed the Q&A pair
        embedding = self.embedder.embed_query(f"Question: {question} Answer: {answer}")
        
        # Store with metadata
        self.memory_collection.upsert(
            ids=[mem_id],
            embeddings=[embedding],
            documents=[content],
            metadatas=[{
                "question": question,
                "answer": answer[:500],  # Truncate long answers
                "timestamp": datetime.now().isoformat(),
                "importance": importance_score
            }]
        )
        return mem_id
    
    def retrieve_relevant_memories(self, query: str, n_results: int = 3) -> List[Dict]:
        """
        Semantic search over LONG-TERM memory for relevant past conversations
        """
        query_embedding = self.embedder.embed_query(query)
        results = self.memory_collection.query(
            query_embeddings=[query_embedding],
            n_results=n_results
        )
        
        memories = []
        if results['metadatas'] and results['metadatas'][0]:
            for meta in results['metadatas'][0]:
                memories.append({
                    "question": meta.get("question", ""),
                    "answer": meta.get("answer", ""),
                    "timestamp": meta.get("timestamp", "")
                })
        
        return memories
    
    def extract_and_store_facts(self, text: str, source: str = "conversation"):
        """
        Extract key facts from text and store in dedicated facts collection
        (Placeholder for future LLM-based fact extraction)
        """
        # Simple implementation: store as-is with embedding
        fact_id = hashlib.md5(f"{text}{datetime.now()}".encode()).hexdigest()
        embedding = self.embedder.embed_query(text)
        
        self.facts_collection.upsert(
            ids=[fact_id],
            embeddings=[embedding],
            documents=[text],
            metadatas=[{
                "source": source,
                "timestamp": datetime.now().isoformat()
            }]
        )
    
    def consolidate_memory(self, llm_summarizer=None):
        """
        Consolidate old short-term memories into long-term storage
        Called periodically or when buffer is full
        """
        if len(self.conversation_buffer) >= self.max_short_term - 1:
            # Store oldest exchanges to long-term
            for exch in list(self.conversation_buffer)[:3]:  # Store oldest 3
                self.store_to_long_term(
                    exch['question'], 
                    exch['answer'], 
                    importance_score=0.6  # Default importance
                )
            print("🗄️ Consolidated 3 exchanges to long-term memory")
    
    def clear_short_term(self):
        """Reset short-term memory (new session)"""
        self.conversation_buffer.clear()
        print("🧹 Short-term memory cleared")

print("✅ MemoryManager class defined")

✅ MemoryManager class defined


In [18]:
# Initialize Memory Manager with the engine's ChromaDB client
memory_manager = MemoryManager(
    chromadb_client=engine.client,
    embedder=engine.dense_embedder,
    max_short_term=10  # Keep last 10 Q&A pairs in short-term memory
)

print("🧠 Memory system ready for use")

✅ Memory Manager Initialized
   - Short-term capacity: 10 exchanges
   - Long-term storage: ChromaDB (persistent)
🧠 Memory system ready for use


In [20]:
import json
import ast
from langgraph.graph import StateGraph, END
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# --- 1. ROBUST PARSING HELPER ---
def parse_json_safe(text_output):
    """
    Safely parses LLM output that might use single quotes or markdown blocks.
    """
    # Clean markdown
    text = text_output.strip().replace("```json", "").replace("```", "")
    
    try:
        # Try standard JSON
        return json.loads(text)
    except json.JSONDecodeError:
        try:
            # Try Python literal eval (handles single quotes)
            return ast.literal_eval(text)
        except:
            # Fallback: simple string search
            if "yes" in text.lower(): return {"score": "yes"}
            return {"score": "no"}

# --- 2. MEMORY-AUGMENTED PROMPTS & CHAINS ---

# A. Memory Retrieval Prompt (NEW)
memory_retrieval_prompt = PromptTemplate(
    template="""Based on the current question, determine if we need context from previous conversations.
    
    Current Question: {question}
    
    Should we retrieve conversation history? Return JSON: {{"retrieve": "yes"}} or {{"retrieve": "no"}}""",
    input_variables=["question"]
)
memory_decision_chain = memory_retrieval_prompt | llm_router | StrOutputParser()

# B. RAG Chain with Memory Context (UPDATED)
rag_with_memory_prompt = PromptTemplate(
    template="""You are a helpful assistant with access to document context and conversation history.
    
    📚 DOCUMENT CONTEXT:
    {context}
    
    🧠 CONVERSATION HISTORY:
    {memory_context}
    
    🧠 RELEVANT PAST CONVERSATIONS:
    {long_term_memory}
    
    ❓ CURRENT QUESTION: {question}
    
    Instructions:
    - Use the document context as your primary source
    - Reference conversation history to maintain context continuity
    - If the question refers to previous answers, use the memory
    - Be concise and accurate
    
    Answer:""",
    input_variables=["context", "memory_context", "long_term_memory", "question"]
)
rag_chain = rag_with_memory_prompt | llm_gen | StrOutputParser()

# C. Rewriter (Same)
rewrite_prompt = PromptTemplate(
    template="""You are a search query optimizer. 
    Rewrite the user's question to be short, specific, and keyword-rich for a vector database.
    Do NOT output a list. Output ONLY the rewritten query string.
    
    Original: {question}
    New Query:""",
    input_variables=["question"]
)
rewriter_chain = rewrite_prompt | llm_router | StrOutputParser()

# D. Grader (Same)
grade_prompt = PromptTemplate(
    template="""You are a grader assessing relevance. 
    Does the document contain ANY information related to the user's question?
    
    Document: {document}
    Question: {question}
    
    Return strictly valid JSON: {{"score": "yes"}} or {{"score": "no"}}.""",
    input_variables=["question", "document"]
)
grader_chain = grade_prompt | llm_router | StrOutputParser()

# E. Grounding (Same)
grounding_prompt = PromptTemplate(
    template="""Context: {context} 
    Answer: {generation} 
    
    Is the answer fully supported by the context? 
    Return strictly valid JSON: {{"score": "yes"}} or {{"score": "no"}}.""",
    input_variables=["context", "generation"]
)
grounding_chain = grounding_prompt | llm_router | StrOutputParser()

# --- 3. MEMORY-AUGMENTED GRAPH DEFINITION ---

class AgentState(TypedDict):
    question: str
    original_question: str  # NEW: Keep original for memory storage
    documents: List[str]
    generation: str
    is_grounded: bool
    retries: int
    memory_context: str  # NEW: Short-term memory
    long_term_memory: str  # NEW: Retrieved long-term memories
    should_store_memory: bool  # NEW: Flag for memory storage

# --- GRAPH NODES ---

def memory_retrieval_node(state):
    """NEW: Retrieve relevant memories before processing"""
    print("🧠 Retrieving Relevant Memories...")
    
    # Get short-term memory (recent conversation)
    recent_history = memory_manager.format_history_for_prompt(n=3)
    
    # Get long-term memory (semantic search over past conversations)
    long_term = memory_manager.retrieve_relevant_memories(state["question"], n_results=2)
    long_term_formatted = "\n".join([
        f"Past Q: {m['question']}\nPast A: {m['answer'][:200]}..." 
        for m in long_term
    ]) if long_term else "No relevant past conversations found."
    
    return {
        "memory_context": recent_history,
        "long_term_memory": long_term_formatted,
        "original_question": state.get("original_question") or state["question"]
    }

def retrieve_node(state):
    print(f"🔍 Hybrid Retrieval for: '{state['question']}'")
    docs = engine.hybrid_search(state["question"])
    return {"documents": docs, "retries": state.get("retries", 0)}

def grade_node(state):
    print("📝 Grading Retrieval Quality...")
    relevant = []
    
    if not state["documents"]: 
        return {"documents": [], "retries": state["retries"] + 1}
        
    for doc in state["documents"]:
        try:
            raw_res = grader_chain.invoke({"question": state["question"], "document": doc})
            parsed_res = parse_json_safe(raw_res)
            
            if parsed_res.get("score") == "yes":
                relevant.append(doc)
        except: continue
    
    new_retries = state["retries"] + 1 if not relevant else state["retries"]
    return {"documents": relevant, "retries": new_retries}

def rewrite_node(state):
    print("🔄 Retrieval Poor. Rewriting Query...")
    new_q = rewriter_chain.invoke({"question": state["question"]})
    new_q = new_q.strip().replace('"', '')
    print(f"   -> New Query: '{new_q}'")
    return {"question": new_q}

def generate_node(state):
    print("💡 Generating Answer with Memory Context...")
    gen = rag_chain.invoke({
        "context": state["documents"], 
        "question": state["question"],
        "memory_context": state.get("memory_context", ""),
        "long_term_memory": state.get("long_term_memory", "")
    })
    return {"generation": gen, "should_store_memory": True}

def graceful_fail_node(state):
    print("❌ Max Retries Reached. Returning fallback.")
    return {
        "generation": "I'm sorry, I tried several times but couldn't find relevant information in the provided document to answer your question. Please try rephrasing.",
        "should_store_memory": False
    }

def reflection_node(state):
    print("🛡️ Verifying Grounding...")
    if "I'm sorry" in state["generation"]:
        return {"is_grounded": False}
        
    raw_res = grounding_chain.invoke({"context": state["documents"], "generation": state["generation"]})
    parsed_res = parse_json_safe(raw_res)
    
    if parsed_res.get("score") == "no":
        return {
            "is_grounded": False, 
            "generation": "I found some documents, but the answer I generated couldn't be fully verified against them.",
            "should_store_memory": False
        }
    return {"is_grounded": True}

def memory_storage_node(state):
    """NEW: Store successful Q&A to memory"""
    print("💾 Storing to Memory...")
    
    if state.get("should_store_memory") and state.get("is_grounded"):
        # Add to short-term memory
        memory_manager.add_exchange(
            question=state["original_question"],
            answer=state["generation"],
            metadata={"grounded": True}
        )
        
        # Conditionally store to long-term (important conversations)
        # You can add logic here to determine importance
        importance = 0.7 if len(state["generation"]) > 100 else 0.5
        memory_manager.store_to_long_term(
            question=state["original_question"],
            answer=state["generation"],
            importance_score=importance
        )
        
        # Consolidate if buffer is full
        memory_manager.consolidate_memory()
        print("   ✅ Memory stored successfully")
    
    return {}

# --- BUILD GRAPH ---
workflow = StateGraph(AgentState)

# Add all nodes
workflow.add_node("memory_retrieval", memory_retrieval_node)  # NEW
workflow.add_node("retrieve", retrieve_node)
workflow.add_node("grade", grade_node)
workflow.add_node("rewrite", rewrite_node)
workflow.add_node("generate", generate_node)
workflow.add_node("reflect", reflection_node)
workflow.add_node("memory_storage", memory_storage_node)  # NEW
workflow.add_node("fail", graceful_fail_node)

# Set entry point - START WITH MEMORY RETRIEVAL
workflow.set_entry_point("memory_retrieval")
workflow.add_edge("memory_retrieval", "retrieve")
workflow.add_edge("retrieve", "grade")

def check_relevance(state):
    if state["documents"]: return "generate"
    if state["retries"] > 2: return "fail"
    return "rewrite"

workflow.add_conditional_edges("grade", check_relevance)
workflow.add_edge("rewrite", "retrieve")
workflow.add_edge("generate", "reflect")

# After reflection, store memory before ending
def check_reflection(state):
    return "memory_storage"

workflow.add_edge("reflect", "memory_storage")
workflow.add_edge("memory_storage", END)
workflow.add_edge("fail", END)

app = workflow.compile()
print("✅ Memory-Augmented Agent Graph Compiled")
print("   🧠 Short-term: Conversation buffer (in-memory)")
print("   🗄️ Long-term: ChromaDB persistent storage")
print("   ♻️ Auto-consolidation enabled")

✅ Memory-Augmented Agent Graph Compiled
   🧠 Short-term: Conversation buffer (in-memory)
   🗄️ Long-term: ChromaDB persistent storage
   ♻️ Auto-consolidation enabled


In [21]:
if 'engine' in globals() and 'memory_manager' in globals():
    q = "What is the consolidated revenue growth?"
    print(f"\n❓ User: {q}")
    
    # Invoke the memory-augmented app
    res = app.invoke({
        "question": q, 
        "retries": 0,
        "memory_context": "",
        "long_term_memory": "",
        "should_store_memory": False
    })
    
    print("\n" + "="*40)
    print(f"✅ Final Answer:\n{res.get('generation', 'No answer generated.')}")
    print("="*40)
    
    # Show memory status
    print(f"\n🧠 Short-term memory: {len(memory_manager.conversation_buffer)} exchanges")
else:
    print("❌ Engine or Memory Manager not found. Please run previous cells first.")


❓ User: What is the consolidated revenue growth?
🧠 Retrieving Relevant Memories...
🔍 Hybrid Retrieval for: 'What is the consolidated revenue growth?'
📝 Grading Retrieval Quality...
💡 Generating Answer with Memory Context...
🛡️ Verifying Grounding...
💾 Storing to Memory...
   ✅ Memory stored successfully

✅ Final Answer:
The consolidated revenue growth is 6.5%, with revenue from operations increasing to ₹117,055 crores in the year ended 31 March 2025 from ₹109,913 crores in the year ended 31 March 2024.

🧠 Short-term memory: 1 exchanges


In [22]:
# 🔧 MEMORY MANAGEMENT UTILITIES

def show_memory_stats():
    """Display comprehensive memory statistics"""
    print("📊 MEMORY SYSTEM STATISTICS")
    print("="*50)
    
    # Short-term
    print(f"\n🧠 SHORT-TERM MEMORY:")
    print(f"   - Buffer size: {len(memory_manager.conversation_buffer)}/{memory_manager.max_short_term}")
    print(f"   - Capacity: {(len(memory_manager.conversation_buffer)/memory_manager.max_short_term)*100:.1f}%")
    
    # Long-term
    long_term_count = memory_manager.memory_collection.count()
    facts_count = memory_manager.facts_collection.count()
    
    print(f"\n🗄️ LONG-TERM MEMORY:")
    print(f"   - Stored conversations: {long_term_count}")
    print(f"   - Extracted facts: {facts_count}")
    print(f"   - Total persistent records: {long_term_count + facts_count}")

def search_memory(query: str, n=3):
    """Search through long-term memory"""
    print(f"\n🔍 Searching memory for: '{query}'")
    memories = memory_manager.retrieve_relevant_memories(query, n_results=n)
    
    if memories:
        print(f"\n✅ Found {len(memories)} relevant memories:\n")
        for i, mem in enumerate(memories, 1):
            print(f"[{i}] Q: {mem['question']}")
            print(f"    A: {mem['answer'][:150]}...")
            print(f"    Time: {mem['timestamp']}")
            print()
    else:
        print("❌ No relevant memories found")

def clear_all_memory():
    """Reset all memory (use with caution!)"""
    confirm = input("⚠️ This will clear ALL memory. Type 'YES' to confirm: ")
    if confirm == "YES":
        memory_manager.clear_short_term()
        # Note: ChromaDB collections persist - would need manual deletion
        print("✅ Short-term memory cleared. Long-term memory persists in ChromaDB.")
    else:
        print("❌ Operation cancelled")

def export_conversation_history():
    """Export current conversation to dict"""
    history = memory_manager.get_recent_history()
    print(f"\n📤 Exporting {len(history)} exchanges...")
    return history

# Demo usage
if 'memory_manager' in globals():
    show_memory_stats()
    print("\n" + "="*50)
    print("Available utilities:")
    print("  - show_memory_stats()")
    print("  - search_memory('your query', n=3)")
    print("  - clear_all_memory()")
    print("  - export_conversation_history()")
else:
    print("❌ Memory Manager not initialized")

📊 MEMORY SYSTEM STATISTICS

🧠 SHORT-TERM MEMORY:
   - Buffer size: 1/10
   - Capacity: 10.0%

🗄️ LONG-TERM MEMORY:
   - Stored conversations: 1
   - Extracted facts: 0
   - Total persistent records: 1

Available utilities:
  - show_memory_stats()
  - search_memory('your query', n=3)
  - clear_all_memory()
  - export_conversation_history()


In [24]:
# 🧠 MEMORY-AUGMENTED MULTI-TURN CONVERSATION TEST
print("🚀 Starting Memory-Augmented Conversation Demo...\n")

if 'app' in globals() and 'engine' in globals() and 'memory_manager' in globals():
    
    # Test conversation flow that demonstrates memory
    conversation_flow = [
        "What is the trend of attrition rate over the last 3 years?",
        "Can you elaborate on the first point you mentioned?",  # Memory reference
        "What about revenue distribution across geographies?",
        "How does that compare to what you said earlier about attrition?",  # Cross-reference
        "Summarize everything we've discussed so far"  # Memory synthesis
    ]
    
    for i, q in enumerate(conversation_flow, 1):
        print(f"\n{'='*60}")
        print(f"❓ Question {i}: {q}")
        print('='*60)
        
        try:
            # Invoke the Agent
            res = app.invoke({
                "question": q,
                "retries": 0,
                "memory_context": "",
                "long_term_memory": "",
                "should_store_memory": False
            })
            
            answer = res.get("generation", "No answer generated.")
            
            print(f"\n✅ Agent Answer:\n{answer}")
            
            # Show memory state after each exchange
            print(f"\n📊 Memory Status:")
            print(f"   - Short-term buffer: {len(memory_manager.conversation_buffer)} exchanges")
            print(f"   - Memory stored: {'✓' if res.get('should_store_memory') else '✗'}")
            
        except Exception as e:
            print(f"❌ Error: {e}")
        
        print("\n" + "-"*60)
    
    # Final memory summary
    print("\n" + "="*60)
    print("📚 FINAL MEMORY SUMMARY")
    print("="*60)
    print(f"\n🧠 Short-Term Memory ({len(memory_manager.conversation_buffer)} exchanges):")
    for idx, exch in enumerate(memory_manager.get_recent_history(), 1):
        print(f"\n[{idx}] Q: {exch['question']}")
        print(f"    A: {exch['answer'][:100]}...")
    
else:
    print("❌ Agent app, Engine, or Memory Manager not found. Please ensure previous cells were run successfully.")

🚀 Starting Memory-Augmented Conversation Demo...


❓ Question 1: What is the trend of attrition rate over the last 3 years?
🧠 Retrieving Relevant Memories...
🔍 Hybrid Retrieval for: 'What is the trend of attrition rate over the last 3 years?'
📝 Grading Retrieval Quality...
💡 Generating Answer with Memory Context...
🛡️ Verifying Grounding...
💾 Storing to Memory...

✅ Agent Answer:
I found some documents, but the answer I generated couldn't be fully verified against them.

📊 Memory Status:
   - Short-term buffer: 4 exchanges
   - Memory stored: ✗

------------------------------------------------------------

❓ Question 2: Can you elaborate on the first point you mentioned?
🧠 Retrieving Relevant Memories...
🔍 Hybrid Retrieval for: 'Can you elaborate on the first point you mentioned?'
📝 Grading Retrieval Quality...
💡 Generating Answer with Memory Context...
🛡️ Verifying Grounding...
💾 Storing to Memory...
   ✅ Memory stored successfully

✅ Agent Answer:
This conversation has just begun. I 

In [23]:
# 🎨 VISUAL MEMORY FLOW DEMO
# Run this cell to see the complete memory lifecycle in action

def demo_memory_lifecycle():
    """
    Demonstrates the complete memory lifecycle with visual feedback
    """
    print("╔" + "="*60 + "╗")
    print("║" + " "*15 + "MEMORY LIFECYCLE DEMO" + " "*24 + "║")
    print("╚" + "="*60 + "╝\n")
    
    if 'memory_manager' not in globals():
        print("❌ Memory Manager not initialized. Run previous cells first.")
        return
    
    # Step 1: Show initial state
    print("📊 STEP 1: Initial Memory State")
    print("-" * 60)
    print(f"   Short-term buffer: {len(memory_manager.conversation_buffer)} exchanges")
    print(f"   Long-term storage: {memory_manager.memory_collection.count()} records")
    
    # Step 2: Simulate adding memories
    print("\n💾 STEP 2: Adding Sample Exchanges to Short-Term")
    print("-" * 60)
    sample_exchanges = [
        ("What is the company revenue?", "Total revenue is $500M in FY24"),
        ("How about profit margins?", "Net profit margin is 15%"),
        ("What are growth projections?", "Projected 20% YoY growth for next 3 years")
    ]
    
    for q, a in sample_exchanges:
        memory_manager.add_exchange(q, a, {"demo": True})
        print(f"   ✓ Added: Q: {q[:40]}...")
    
    print(f"\n   Buffer now contains: {len(memory_manager.conversation_buffer)} exchanges")
    
    # Step 3: Retrieve and format
    print("\n🔍 STEP 3: Retrieve Recent History (for LLM context)")
    print("-" * 60)
    formatted = memory_manager.format_history_for_prompt(n=3)
    print(formatted)
    
    # Step 4: Store to long-term
    print("\n🗄️ STEP 4: Persist Important Exchange to Long-Term")
    print("-" * 60)
    mem_id = memory_manager.store_to_long_term(
        question=sample_exchanges[0][0],
        answer=sample_exchanges[0][1],
        importance_score=0.85
    )
    print(f"   ✓ Stored with ID: {mem_id[:16]}...")
    print(f"   Long-term storage: {memory_manager.memory_collection.count()} records")
    
    # Step 5: Semantic search
    print("\n🔎 STEP 5: Semantic Search Over Long-Term Memory")
    print("-" * 60)
    search_query = "financial performance"
    print(f"   Query: '{search_query}'")
    memories = memory_manager.retrieve_relevant_memories(search_query, n_results=2)
    
    if memories:
        print(f"   ✓ Found {len(memories)} relevant memories:")
        for i, mem in enumerate(memories, 1):
            print(f"\n   [{i}] Q: {mem['question']}")
            print(f"       A: {mem['answer'][:60]}...")
    else:
        print("   (No matches - run after actual conversations)")
    
    # Step 6: Consolidation
    print("\n♻️ STEP 6: Auto-Consolidation (when buffer full)")
    print("-" * 60)
    print(f"   Current capacity: {len(memory_manager.conversation_buffer)}/{memory_manager.max_short_term}")
    print(f"   Trigger point: {memory_manager.max_short_term - 1}")
    print("   Status: " + ("⚠️ Will consolidate soon" if len(memory_manager.conversation_buffer) >= memory_manager.max_short_term - 2 else "✅ Within limits"))
    
    print("\n" + "╔" + "="*60 + "╗")
    print("║" + " "*18 + "LIFECYCLE COMPLETE" + " "*24 + "║")
    print("╚" + "="*60 + "╝\n")
    
    print("💡 Key Takeaways:")
    print("   1. Short-term memory = Fast, recent context (in-memory)")
    print("   2. Long-term memory = Persistent, searchable (ChromaDB)")
    print("   3. Auto-consolidation prevents buffer overflow")
    print("   4. Semantic search enables intelligent context retrieval")

# Run the demo
demo_memory_lifecycle()

╔============================================================╗
║               MEMORY LIFECYCLE DEMO                        ║
╚============================================================╝

📊 STEP 1: Initial Memory State
------------------------------------------------------------
   Short-term buffer: 1 exchanges
   Long-term storage: 1 records

💾 STEP 2: Adding Sample Exchanges to Short-Term
------------------------------------------------------------
   ✓ Added: Q: What is the company revenue?...
   ✓ Added: Q: How about profit margins?...
   ✓ Added: Q: What are growth projections?...

   Buffer now contains: 4 exchanges

🔍 STEP 3: Retrieve Recent History (for LLM context)
------------------------------------------------------------
[1] Q: What is the company revenue?
    A: Total revenue is $500M in FY24...
[2] Q: How about profit margins?
    A: Net profit margin is 15%...
[3] Q: What are growth projections?
    A: Projected 20% YoY growth for next 3 years...

🗄️ STEP 4: Persist 